<a href="https://colab.research.google.com/github/slowandfast/DiagRecommdSystem_01/blob/main/KJH_BERT_EMR%EA%B8%B0%EB%B0%98_%EC%A7%84%EB%8B%A8%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [21]:
file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'
df = pd.read_excel(file_path)
df.head(5)

,ICD10,진단코드,한글진단명,영문진단명,나이,성별,기록작성일,작성과,작성의사,주호소,현병력,현병력2,transstr,과거력
0,N391,N391.000.01,Persistent proteinuria,지속성 단백뇨증,51,F,20230703,신장내과,NPN,NaN,115-61-61_x000D_\n2023.5.10._x000D_\n_x000D_\n...,115-61-61 2023.5.10. 검진상 단백뇨 의심 소견 + 거품뇨 ...,115-61-61 2023.5.10.Suspicion of proteinuria i...,자궁적출_x000D_\n_x000D_\n당뇨 고혈압 없음_x000D_\n_x000D...
1,N184,N184.001.00,"Chronic kidney disease, stage 4",만성 신장질환(4기),79,M,20230703,신장내과,NPP,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69_x000D_\n요증상(-), edema(-), 요독증상(-), D...","130/61/69 요증상(-), edema(-), 요독증상(-), DOE(-) ...","130/61/69 Symptoms (-), edema (-), urbanic sym...",DM/HTN (-/+)_x000D_\nHerb med hx (-): 영양제 복용중임...
2,N184,N184.001.00,"Chronic kidney disease, stage 4",만성 신장질환(4기),71,F,20230703,신장내과,NPP,CKD4,"153/75/85_x000D_\n요증상(-), edema(-), 요독증상(-), D...","153/75/85 요증상(-), edema(-), 요독증상(-), DOE(-) ...","153/75/85 osophy (-), edema (-), urinary sympt...",DM/HTN (+/+)_x000D_\nHerb med hx (-)_x000D_\nA...
3,N185,N185.011.00,"Chronic kidney disease, stage 5",만성 신장질환(5기),52,F,20230703,신장내과,NPP,"Cr 3.74, eGFR 13.56","148/90/64_x000D_\n요증상(-), edema mild_x000D_\nC...","148/90/64 요증상(-), edema mild CKD d/t ADPKD로 ...",148/90/64 Idema Mild CKD D/T ADPKD has been tr...,DM/HTN (-/+)_x000D_\nHerb med hx (-)_x000D_\nA...
4,N183,N183.001.00,"Chronic kidney disease, stage 3",만성 신장질환(3기),82,F,20230703,신장내과,NPK,lab abnormality,선생님 안녕하십니까_x000D_\n_x000D_\n상기 병명으로 입원하여 치료중인 ...,선생님 안녕하십니까 상기 병명으로 입원하여 치료중인 환자입니다. 환자 입...,"Hello teacher, I am a patient who is being hos...","# Cerebral infarction, Lt MCA territory (F cor..."


In [41]:
# 전처리 1
df['주호소2'] = df['주호소'].str.replace("_x000D_", " ")
df['주호소2'] = df['주호소2'].str.replace("\n", " ")
df['주호소2'] = df['주호소2'].str.strip()

df['현병력2'] = df['현병력'].str.replace("_x000D_", " ")
df['현병력2'] = df['현병력2'].str.replace("\n", " ")
df['현병력2'] = df['현병력2'].str.strip()

df['과거력2'] = df['과거력'].str.replace("_x000D_", " ")
df['과거력2'] = df['과거력2'].str.replace("\n", " ")
df['과거력2'] = df['과거력2'].str.strip()

In [42]:
df.head(100)

,ICD10,진단코드,한글진단명,영문진단명,나이,성별,기록작성일,작성과,작성의사,주호소,현병력,현병력2,transstr,과거력,주호소2,과거력2
0,N391,N391.000.01,Persistent proteinuria,지속성 단백뇨증,51,F,20230703,신장내과,NPN,NaN,115-61-61_x000D_\n2023.5.10._x000D_\n_x000D_\n...,115-61-61 2023.5.10. 검진상 단백뇨 의심 소견 + 거품뇨 ...,115-61-61 2023.5.10.Suspicion of proteinuria i...,자궁적출_x000D_\n_x000D_\n당뇨 고혈압 없음_x000D_\n_x000D...,NaN,자궁적출 당뇨 고혈압 없음 no medication
1,N184,N184.001.00,"Chronic kidney disease, stage 4",만성 신장질환(4기),79,M,20230703,신장내과,NPP,"CKD4, Cr 2.26, eGFR 29ml/min","130/61/69_x000D_\n요증상(-), edema(-), 요독증상(-), D...","130/61/69 요증상(-), edema(-), 요독증상(-), DOE(-) ...","130/61/69 Symptoms (-), edema (-), urbanic sym...",DM/HTN (-/+)_x000D_\nHerb med hx (-): 영양제 복용중임...,"CKD4, Cr 2.26, eGFR 29ml/min",DM/HTN (-/+) Herb med hx (-): 영양제 복용중임. Alco...
2,N184,N184.001.00,"Chronic kidney disease, stage 4",만성 신장질환(4기),71,F,20230703,신장내과,NPP,CKD4,"153/75/85_x000D_\n요증상(-), edema(-), 요독증상(-), D...","153/75/85 요증상(-), edema(-), 요독증상(-), DOE(-) ...","153/75/85 osophy (-), edema (-), urinary sympt...",DM/HTN (+/+)_x000D_\nHerb med hx (-)_x000D_\nA...,CKD4,DM/HTN (+/+) Herb med hx (-) Alcohol/smoking...
3,N185,N185.011.00,"Chronic kidney disease, stage 5",만성 신장질환(5기),52,F,20230703,신장내과,NPP,"Cr 3.74, eGFR 13.56","148/90/64_x000D_\n요증상(-), edema mild_x000D_\nC...","148/90/64 요증상(-), edema mild CKD d/t ADPKD로 ...",148/90/64 Idema Mild CKD D/T ADPKD has been tr...,DM/HTN (-/+)_x000D_\nHerb med hx (-)_x000D_\nA...,"Cr 3.74, eGFR 13.56",DM/HTN (-/+) Herb med hx (-) Alcohol/smoking...
4,N183,N183.001.00,"Chronic kidney disease, stage 3",만성 신장질환(3기),82,F,20230703,신장내과,NPK,lab abnormality,선생님 안녕하십니까_x000D_\n_x000D_\n상기 병명으로 입원하여 치료중인 ...,선생님 안녕하십니까 상기 병명으로 입원하여 치료중인 환자입니다. 환자 입...,"Hello teacher, I am a patient who is being hos...","# Cerebral infarction, Lt MCA territory (F cor...",lab abnormality,"# Cerebral infarction, Lt MCA territory (F cor..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,N178,N178.011.00,"Acute kidney injury(AKI), Nontraumatic",급성 신손상(비외상성),62,M,20230719,신장내과,NPH,azotemia,안녕하십니까 교수님._x000D_\n_x000D_\n# CM on lt. o...,안녕하십니까 교수님. # CM on lt. occipital lobe with...,"Hello, Professor.# Cm on lt.OCCIPITAL LOBE wit...",NaN,azotemia,NaN
96,N178,N178.011.00,"Acute kidney injury(AKI), Nontraumatic",급성 신손상(비외상성),80,M,20230719,신장내과,NPH,consultation for operation,상기 환자는 진단명[Hyperplasia of prostate with ob...,상기 환자는 진단명[Hyperplasia of prostate with obstru...,The patient is a 79 -year -old (m) patient cur...,NaN,consultation for operation,NaN
97,N183,N183.001.00,"Chronic kidney disease, stage 3",만성 신장질환(3기),71,M,20230719,신장내과,NPH,consultation for contrast use,"상기 환자는 진단명[Aneurysm, P-comm](으)로 현재 치료중인 7...","상기 환자는 진단명[Aneurysm, P-comm](으)로 현재 치료중인 70세(M...",The patient is a 70-year-old patient currently...,NaN,consultation for contrast use,NaN
98,N185,N185.007.00,End stage kidney disease on dialysis,말기 신장병 투석중,37,M,20230719,신장내과,NPH,KONOS 등록,"_x000D_\nkonos f/u_x000D_\nHD 주3회 월,수,금 at 영천요...","konos f/u HD 주3회 월,수,금 at 영천요양병원 Dwt 92.8...","Konos F/U HD 3 times a week, Wed, Gold AT Yeon...",NaN,KONOS 등록,NaN


In [3]:
# file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'

# selected_columns = ['ICD10', 'transstr']
# df = pd.read_excel(file_path, usecols=selected_columns)

# print('전체 데이터 수 :',len(df))
# df.head(5)

전체 데이터 수 : 826


,ICD10,transstr
0,N391,115-61-61 2023.5.10.Suspicion of proteinuria i...
1,N184,"130/61/69 Symptoms (-), edema (-), urbanic sym..."
2,N184,"153/75/85 osophy (-), edema (-), urinary sympt..."
3,N185,148/90/64 Idema Mild CKD D/T ADPKD has been tr...
4,N183,"Hello teacher, I am a patient who is being hos..."


In [1]:
pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 976.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=d3c1391ff70b1fbac94f57d72974686329903df1c27a92b8c10e38b6aedc542b
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstallin

In [46]:
# 번역
import googletrans

def trans_hng_to_eng(text):
    rtn = ''
    # print('text : ', text )
    try:
        if text is not None:
            translator = googletrans.Translator()
            text = translator.translate(text, dest='en')
            # print('T : ', text.text)
            if text is not None:
                rtn = text.text
    except:

    return rtn


KeyboardInterrupt: 

In [47]:
# df['CC_translated'] = df['주호소2'].apply(trans_hng_to_eng)
# df['PI_translated'] = df['현병력2'].apply(trans_hng_to_eng)
df['PH_translated'] = df['과거력2'].apply(trans_hng_to_eng)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
# df['CC_translated']
# df['PI_translated']
# df['PH_translated']

In [49]:
df.to_excel('/content/translated_ds.xlsx')

In [4]:
# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(df['transstr'], df['ICD10'], test_size=0.2, random_state=5)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=7)

print(X_train)
print("=======================================\n")
print(X_test)
print("=======================================\n")
print(X_val)
print("=======================================\n")
print(Y_train)
print("=======================================\n")
print(Y_test)
print("=======================================\n")
print(Y_val)

443    2021.09 CABG surgery 2023.06 CR 4.0, EGFR 14 2...
441    Since 2016, Hematuria had hematuria in 2022 TY...
66     23.04 LMC obstetrics and gynecology visits Hem...
773    60s HTN Diagnosis 2023.11.21 CR 1.58 BP 121/67...
131    I didn't have a previous checkup.2023. CR 1.83...
                             ...                        
73     156/94/83, HOME BP <130/80, and Type A told fo...
400    # HTN, On Medication.Comprehensive Award at th...
118    The patient is a patient of 72 years old, curr...
701    HTN, DM 25 -year LMC Drug During LMC Pharmaceu...
206    The patient is a 69 -year -old (m) patient cur...
Name: transstr, Length: 660, dtype: object

278    114-69-88 2 years ago Urology and Browno Holy ...
690    97-53-112 Diabetes and Hypertension-more than ...
108    145-75-85 2023.4.Microscopic Hematuria 2022.10...
499    2023/04 I heard that there is proteinuria duri...
755    The patient is a patient of 77 years old, curr...
                             ...            

In [ ]:
#https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [6]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('NitzanBar/umls-spanbert')

# Model
model = AutoModelForSequenceClassification.from_pretrained('NitzanBar/umls-spanbert')
model.to(device)

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [11]:
# tokenizer 토큰화 결과 확인
result = tokenizer.tokenize("130/61/69 Symptoms (-), edema (-), urbanic symptoms (-), doe (-) Known htn (> 10YRS), hypothyroidism, DL patient with LMC inspection CR 2.26, EGFR 29, HB 12.7Visit as.")
print(result)
print(tokenizer.vocab['symptoms'])
print([tokenizer.encode(token) for token in result])

['130', '/', '61', '/', '69', 'symptoms', '(', '-', ')', ',', 'ed', '##ema', '(', '-', ')', ',', 'urban', '##ic', 'symptoms', '(', '-', ')', ',', 'do', '##e', '(', '-', ')', 'known', 'h', '##t', '##n', '(', '>', '10', '##yr', '##s', ')', ',', 'h', '##y', '##pot', '##hy', '##roid', '##ism', ',', 'd', '##l', 'patient', 'with', 'l', '##m', '##c', 'inspection', 'c', '##r', '2', '.', '26', ',', 'e', '##g', '##f', '##r', '29', ',', 'h', '##b', '12', '.', '7', '##vis', '##it', 'as', '.']
8006
[[101, 7029, 102], [101, 120, 102], [101, 5391, 102], [101, 120, 102], [101, 5691, 102], [101, 8006, 102], [101, 113, 102], [101, 118, 102], [101, 114, 102], [101, 117, 102], [101, 5048, 102], [101, 108, 108, 9712, 1161, 102], [101, 113, 102], [101, 118, 102], [101, 114, 102], [101, 117, 102], [101, 3953, 102], [101, 108, 108, 178, 1665, 102], [101, 8006, 102], [101, 113, 102], [101, 118, 102], [101, 114, 102], [101, 117, 102], [101, 1202, 102], [101, 108, 108, 174, 102], [101, 113, 102], [101, 118, 102]

In [12]:
class EMRDiagcdDataset(Dataset):

    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        record = self.df.iloc[index]['transstr']
        # diagcd = self.df.iloc[index]['ICD10']
        diagcd = self.df.iloc[index]['encoded_ICD10']

        return record, diagcd

        # tokens = self.tokenizer(
        #     record,
        #     return_tensors='pt',
        #     truncation=True,
        #     padding='max_length')

        # input_ids = tokens[input_ids].squeeze(0)
        # attention_mask  = tokens[attention_mask ].squeeze(0)
        # token_type_ids  = torch.zeros_like(token_type_ids )

        # return {
        #     'input_ids' : input_ids,
        #     'attention_mask' : attention_mask,
        #     'token_type_ids' : token_type_ids,
        # }, torch.tensor(diagcd)

In [ ]:
# label encoding
unique_label = Y_train.unique()
print("1 :", unique_label)

label_encoder = LabelEncoder()
label_encoder.fit(unique_label)

# print("2 : ", label_encoder)

temp_df = Y_train.to_frame()
temp_df['encoded_ICD10'] = label_encoder.transform(temp_df['ICD10'])
Y_train_encoding = pd.Series(temp_df['encoded_ICD10'])
# Y_train_encoding = Y_train_encoding.rename('encoded_ICD10')
# print("2 :", Y_train_encoding)

1 : ['N185' 'N183' 'R311' 'R601' 'N184' 'N178' 'N391' 'N182' 'N028' 'Z524'
 'R808' 'N029' 'N059' 'R798']


In [ ]:
print(Y_train)

443    N185
441    N183
66     R311
773    N183
131    N183
       ... 
73     Z524
400    R311
118    N183
701    N184
206    N183
Name: ICD10, Length: 660, dtype: object


In [ ]:
print(Y_train_encoding[:5])

[7 5 9 5 5]


In [ ]:
train_df = pd.concat([X_train, Y_train_encoding, Y_train], axis=1)
train_dataset = EMRDiagcdDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
train_df.to_excel('/content/train_df.xlsx')

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-2)
criterion = torch.nn.Softmax()
itr = 1
p_itr = 50
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [ ]:
# train
model.train()

for epoch in range(epochs):
    for record, diagcd in train_loader:
        optimizer.zero_grad()
        # print("record: ", record)
        # print("diagcd: ", diagcd)
        encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
        # encoded_record = encoded_record.to(device)
        # print(type(encoded_record))
        diagcd = torch.tensor(diagcd)
        # diagcd = torch.tensor(list(diagcd), dim=0).to(device)

        outputs = model(**encoded_record, labels=diagcd)

        loss = outputs.loss
        logits = outputs.logits

        predict = torch.argmax(F.softmax(logits), dim=14)
        correct = predict.eq(diagcd)

        total_correct += correct.sum().item()
        total_len += len(diagcd)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr += 1

<ipython-input-118-0be101af0015>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  diagcd = torch.tensor(diagcd)


In [ ]:
test_df = pd.concat([X_test, Y_test], axis=1)
print(test_df)

test_dataset = EMRDiagcdDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)

                                              transstr ICD10
278  114-69-88 2 years ago Urology and Browno Holy ...  R311
690  97-53-112 Diabetes and Hypertension-more than ...  N183
108  145-75-85 2023.4.Microscopic Hematuria 2022.10...  R311
499  2023/04 I heard that there is proteinuria duri...  R311
755  The patient is a patient of 77 years old, curr...  N183
..                                                 ...   ...
228  The patient is a diagnostic name [Paralysis of...  N183
151  # HTN (2022.)2023. CR 1.24 EGFR 41 EGFR K 5.7 ...  N183
129  The patient is a diagnostic [DJD] patient of 8...  N183
781  We will ask for consultation.The patient is a ...  R311
159  20 years ago, DM, HTN diagnosis 10 years ago, ...  N183

[83 rows x 2 columns]


In [ ]:
# evaluation
model.eval()

total_len = 0
total_correct = 0

for record, diagcd in test_loader:

    encoded_record = tokenizer(record, padding=True, truncation=True, max_length=512, return_tensors="pt")
    encoded_record, diagcd = encoded_record.to(device), diagcd.to(device)

    outputs = model(**encoded_record, labels=diagcd)

    logits = outputs.logits

    predict = torch.argmax(F.softmax(logits), dim=14)
    correct = predict.eq(diagcd)

    total_correct += correct.sum().item()
    total_len += len(diagcd)

print('Test accuracy: ', total_correct / total_len)